In [1]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

In [2]:
CODE_DIR = "/home/cbarkhof/fall-2021"
import sys; sys.path.append(CODE_DIR)

# Globals

In [3]:
import torch
if torch.cuda.is_available:
    DEVICE = f"cuda:{torch.cuda.current_device()}"

ANALYSIS_DIR = f"{CODE_DIR}/analysis/analysis-files"
CHECKPOINT_DIR = f"{CODE_DIR}/run_files/checkpoints"

N_SAMPLE_BATCHES = 5
SAMPLE_BATCH_SIZE = 250

BATCH_SIZE_SURPRISAL = 40
N_IW_SAMPLES = 50

KNN_BATCH_SIZE = 100
BATCH_SIZE = 100
NUM_WORKERS = 3

REVERSE = False

# Step 0: Retrieve runs to process

In [4]:
from analysis.MNIST_analysis.analysis_steps import make_run_overview_df

prefixes = ["(mdr-vae-exp 8 oct)", "(fb-vae-exp 8 oct) ", "(beta-vae-exp 6 oct) ", "(inf-vae-exp 5 oct) "]
run_df = make_run_overview_df(prefixes=prefixes)
run_df.drop("run_name", axis=1)

,objective,l_rate,beta_beta,free_bits,mdr_value,l_mmd,decoder
MDR-VAE 40 dec: CNN.T,MDR-VAE,0,0,0,40,0,basic_deconv_decoder
MDR-VAE 32 dec: CNN.T,MDR-VAE,0,0,0,32,0,basic_deconv_decoder
MDR-VAE 24 dec: CNN.T,MDR-VAE,0,0,0,24,0,basic_deconv_decoder
MDR-VAE 16 dec: CNN.T,MDR-VAE,0,0,0,16,0,basic_deconv_decoder
MDR-VAE 8 dec: CNN.T,MDR-VAE,0,0,0,8,0,basic_deconv_decoder
...,...,...,...,...,...,...,...
INFO-VAE l_Rate 100 l_MMD 100 dec: CNN.T,INFO-VAE,100,0,0,0,100,basic_deconv_decoder
INFO-VAE l_Rate 1000 l_MMD 1 dec: CNN.T,INFO-VAE,1000,0,0,0,1,basic_deconv_decoder
INFO-VAE l_Rate 100 l_MMD 1000 dec: PixelCNN++,INFO-VAE,100,0,0,0,1000,cond_pixel_cnn_pp
INFO-VAE l_Rate 1 l_MMD 1000 dec: CNN.T,INFO-VAE,1,0,0,0,1000,basic_deconv_decoder


In [5]:
from analysis.MNIST_analysis.analysis_steps import overview_of_missing_analysis

df_missing = overview_of_missing_analysis(run_df)
display(df_missing.sum(axis=0))
# df_missing

Series([], dtype: float64)

# Step 1: Encode, reconstruct, sample

- `"<ANALYSIS_DIR>/<run_name>/encode-reconstruct-test-valid.pt"`:
    - `'test': "z": [N_t, D], "mean": [N_t, D], "scale": [N_t, D], "reconstruct": [N_t, 1, 28, 28]`
    - `'valid': ,,`
    - `'train': ,,`
<br><br>
- `"<ANALYSIS_DIR>/<run_name>/generative_samples.pt"`:
    - `"z": [N, D], "x": [N, 1, 28, 28]`
<br><br>    
- `"<ANALYSIS_DIR>/<run_name>/generative_conditional_samples.pt"`:
    - `'test': "z": [N_t, D], "mean": [N_t, D], "scale": [N_t, D], "cond_sample_x": [N_t, 1, 28, 28]`
    - `'valid': ,,`
    - `'train': ,,`

In [6]:
from analysis.MNIST_analysis.analysis_steps import encode_reconstruct_sample

encode_reconstruct_sample(run_df, device=DEVICE, include_train=True, 
                          n_sample_batches=N_SAMPLE_BATCHES, 
                          sample_batch_size=SAMPLE_BATCH_SIZE, reverse=REVERSE)

0 (mdr-vae-exp 8 oct) MDR-VAE[R>=40.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-18:30:40
Done this run already, continue
1 (mdr-vae-exp 8 oct) MDR-VAE[R>=32.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:54:06
Done this run already, continue
2 (mdr-vae-exp 8 oct) MDR-VAE[R>=24.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:44:05
Done this run already, continue
3 (mdr-vae-exp 8 oct) MDR-VAE[R>=16.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:37:05
Done this run already, continue
4 (mdr-vae-exp 8 oct) MDR-VAE[R>=8.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:27:34
Done this run already, continue
5 (mdr-vae-exp 8 oct) MDR-VAE[R>=4.0] | q(z|x) independent_ga

Done this run already, continue
44 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=5.0, l_2_mmd=100.0] | q(z|x) independent_gaussian | p(x|z) cond_pixel_cnn_pp | p(z) isotropic_gaussian | D = 10 | 2021-10-05-19:38:36
Done this run already, continue
45 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.5, l_2_mmd=1.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-05-19:38:35
Done this run already, continue
46 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.5, l_2_mmd=100.0] | q(z|x) independent_gaussian | p(x|z) cond_pixel_cnn_pp | p(z) isotropic_gaussian | D = 10 | 2021-10-05-19:38:05
Done this run already, continue
47 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=2.0, l_2_mmd=1.0] | q(z|x) independent_gaussian | p(x|z) cond_pixel_cnn_pp | p(z) isotropic_gaussian | D = 10 | 2021-10-05-19:38:05
Done this run already, continue
48 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.5, l_2_mmd=1.0] | q(z|x) independent_gaussian | p(x|z) cond_pixel_cnn_pp | p(z) isotropic_gaussian

# Step 2: Gather surprisal stats

In [7]:
from analysis.MNIST_analysis.analysis_steps import gather_surprisal_stats

with torch.no_grad():
    gather_surprisal_stats(device=DEVICE, include_train=True, 
                       batch_size_surprisal=BATCH_SIZE_SURPRISAL, n_iw_samples=N_IW_SAMPLES)

0 (mdr-vae-exp 8 oct) MDR-VAE[R>=40.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-18:30:40
mdr_loss: 0.46 | mdr_multiplier: 0.92 | total_loss: 107.70 | mmd: 0.15 | elbo: -107.24 | distortion: 67.73 | kl_prior_post: 39.51 | mean_mean: 0.10 | std_across_x_mean: 1.41 | std_across_z_mean: 1.47 | mean_scale: 0.04 | std_across_x_scale: 0.01 | std_across_z_scale: 0.01 | log_det_cov_q_z: 4.97 | iw_ll: -105.03 | 
did reconstructions already, skipping!
did reconstructions already, skipping!
did samples already, skipping!
did data already, skipping!
1 (mdr-vae-exp 8 oct) MDR-VAE[R>=32.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:54:06
mdr_loss: -0.05 | mdr_multiplier: 0.78 | total_loss: 99.82 | mmd: 0.07 | elbo: -99.87 | distortion: 67.81 | kl_prior_post: 32.06 | mean_mean: -0.06 | std_across_x_mean: 1.15 | std_across_z_mean: 1.14 | mean_scale: 0.05 | std_across_x_

kl_fb: 21.38 | total_loss: 93.43 | mmd: 0.01 | elbo: -93.35 | distortion: 72.05 | kl_prior_post: 21.30 | mean_mean: 0.00 | std_across_x_mean: 1.02 | std_across_z_mean: 1.00 | mean_scale: 0.14 | std_across_x_scale: 0.05 | std_across_z_scale: 0.07 | log_det_cov_q_z: -0.35 | iw_ll: -90.98 | 
did reconstructions already, skipping!
did reconstructions already, skipping!
did samples already, skipping!
did data already, skipping!
16 (fb-vae-exp 8 oct) FB-VAE[4.0-pd=False] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-11:52:39
kl_fb: 20.71 | total_loss: 93.12 | mmd: 0.01 | elbo: -93.10 | distortion: 72.42 | kl_prior_post: 20.68 | mean_mean: 0.01 | std_across_x_mean: 1.01 | std_across_z_mean: 0.99 | mean_scale: 0.15 | std_across_x_scale: 0.07 | std_across_z_scale: 0.08 | log_det_cov_q_z: -0.47 | iw_ll: -90.74 | 
did reconstructions already, skipping!
did reconstructions already, skipping!
did samples already, skipping!
did data alrea

beta_kl: 0.00 | total_loss: 80.76 | mmd: -0.00 | elbo: -80.76 | distortion: 80.76 | kl_prior_post: 0.00 | mean_mean: -0.00 | std_across_x_mean: 0.01 | std_across_z_mean: 0.01 | mean_scale: 1.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | log_det_cov_q_z: -0.93 | iw_ll: -80.68 | 
did reconstructions already, skipping!
did reconstructions already, skipping!
did samples already, skipping!
did data already, skipping!
34 (beta-vae-exp 6 oct) B-VAE[b=2.0] | q(z|x) independent_gaussian | p(x|z) cond_pixel_cnn_pp | p(z) isotropic_gaussian | D = 10 | 2021-10-06-10:00:16
beta_kl: 0.00 | total_loss: 80.84 | mmd: 0.00 | elbo: -80.84 | distortion: 80.84 | kl_prior_post: 0.00 | mean_mean: -0.00 | std_across_x_mean: 0.00 | std_across_z_mean: 0.00 | mean_scale: 1.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | log_det_cov_q_z: -1.05 | iw_ll: -80.76 | 
did reconstructions already, skipping!
did reconstructions already, skipping!
did samples already, skipping!
did data already, sk

lambda_1_Rate: 0.02 | lambda_2_MMD: -0.15 | total_loss: 80.73 | mmd: -0.00 | elbo: -80.87 | distortion: 80.86 | kl_prior_post: 0.00 | mean_mean: -0.00 | std_across_x_mean: 0.00 | std_across_z_mean: 0.02 | mean_scale: 0.99 | std_across_x_scale: 0.00 | std_across_z_scale: 0.01 | log_det_cov_q_z: -1.17 | iw_ll: -80.79 | 
did reconstructions already, skipping!
did reconstructions already, skipping!
did samples already, skipping!
did data already, skipping!
48 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.5, l_2_mmd=1.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-05-19:38:35
lambda_1_Rate: 12.81 | lambda_2_MMD: 0.02 | total_loss: 82.06 | mmd: 0.02 | elbo: -94.85 | distortion: 69.23 | kl_prior_post: 25.62 | mean_mean: 0.01 | std_across_x_mean: 1.07 | std_across_z_mean: 1.04 | mean_scale: 0.09 | std_across_x_scale: 0.04 | std_across_z_scale: 0.04 | log_det_cov_q_z: 0.35 | iw_ll: -92.78 | 
did reconstructions already, skipping!
did recons

lambda_1_Rate: 4.61 | lambda_2_MMD: 0.06 | total_loss: 55.35 | mmd: 0.06 | elbo: -96.77 | distortion: 50.68 | kl_prior_post: 46.09 | mean_mean: -0.13 | std_across_x_mean: 1.16 | std_across_z_mean: 1.17 | mean_scale: 0.01 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | log_det_cov_q_z: 0.98 | iw_ll: -94.71 | 
did reconstructions already, skipping!
did reconstructions already, skipping!
did samples already, skipping!
did data already, skipping!
63 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=100.0, l_2_mmd=100.0] | q(z|x) independent_gaussian | p(x|z) cond_pixel_cnn_pp | p(z) isotropic_gaussian | D = 10 | 2021-10-05-16:38:15
lambda_1_Rate: 0.03 | lambda_2_MMD: -0.07 | total_loss: 80.76 | mmd: -0.00 | elbo: -80.81 | distortion: 80.81 | kl_prior_post: 0.00 | mean_mean: -0.00 | std_across_x_mean: 0.00 | std_across_z_mean: 0.01 | mean_scale: 1.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | log_det_cov_q_z: -1.04 | iw_ll: -80.72 | 
did reconstructions already, skipping!
did recon

lambda_1_Rate: 0.01 | lambda_2_MMD: -0.14 | total_loss: 206.36 | mmd: -0.00 | elbo: -206.49 | distortion: 206.49 | kl_prior_post: 0.00 | mean_mean: -0.00 | std_across_x_mean: 0.00 | std_across_z_mean: 0.00 | mean_scale: 1.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | log_det_cov_q_z: -0.95 | iw_ll: -206.18 | 
did reconstructions already, skipping!
did reconstructions already, skipping!
did samples already, skipping!
did data already, skipping!
77 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=1000.0, l_2_mmd=1.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-05-15:12:26
lambda_1_Rate: 0.01 | lambda_2_MMD: -0.00 | total_loss: 206.42 | mmd: -0.00 | elbo: -206.41 | distortion: 206.41 | kl_prior_post: 0.00 | mean_mean: -0.00 | std_across_x_mean: 0.00 | std_across_z_mean: 0.00 | mean_scale: 1.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | log_det_cov_q_z: -0.97 | iw_ll: -206.28 | 
did reconstructions already, skippin

# Step 3: Gather simple evaluation of validation and test set

In [9]:
from analysis.MNIST_analysis.analysis_steps import simple_evaluate_valid_test

simple_evaluate_valid_test(run_df, device=DEVICE, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

0 (mdr-vae-exp 8 oct) MDR-VAE[R>=40.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-18:30:40
Did this one already, skipping it for now.
1 (mdr-vae-exp 8 oct) MDR-VAE[R>=32.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:54:06
Did this one already, skipping it for now.
2 (mdr-vae-exp 8 oct) MDR-VAE[R>=24.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:44:05
Did this one already, skipping it for now.
3 (mdr-vae-exp 8 oct) MDR-VAE[R>=16.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:37:05
Did this one already, skipping it for now.
4 (mdr-vae-exp 8 oct) MDR-VAE[R>=8.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:27:34
Did this one already, skipping it for now.
5 (mdr

# Step 4: KNN predictions of samples & reconstructions

In [11]:
from analysis.MNIST_analysis.analysis_steps import knn_predictions_for_samples_reconstructions

knn_predictions_for_samples_reconstructions(batch_size=KNN_BATCH_SIZE, 
                                            knn_mimicker_path="/home/cbarkhof/fall-2021/analysis/MNIST_analysis/Notebooks/KNN_mimicking_network.pt",
                                            knn_path=None, device=DEVICE)

0 (mdr-vae-exp 8 oct) MDR-VAE[R>=40.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-18:30:40
1 (mdr-vae-exp 8 oct) MDR-VAE[R>=32.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:54:06
2 (mdr-vae-exp 8 oct) MDR-VAE[R>=24.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:44:05
3 (mdr-vae-exp 8 oct) MDR-VAE[R>=16.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:37:05
4 (mdr-vae-exp 8 oct) MDR-VAE[R>=8.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-17:27:34
5 (mdr-vae-exp 8 oct) MDR-VAE[R>=4.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-08-16:21:06
6 (mdr-vae-exp 8 oct) MDR-VAE[R>=40.0] | q(z|x) independent_gaussi

42 (beta-vae-exp 6 oct) B-VAE[b=1.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-06-08:38:39
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
43 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=2.0, l_2_mmd=1.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-05-19:51:08
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
44 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=0.5, l_2_mmd=100.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-05-19:39:05
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
45 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=5.0, l_2_mmd=1.0] | q(z|x) independent_gaussian | p(x|z) cond_pixel_cnn_pp | p(z) isotropic_gaussian | D = 10 | 2021-10-05-19:38:43
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
46 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=2.

(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
79 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=1.0, l_2_mmd=1000.0] | q(z|x) independent_gaussian | p(x|z) basic_deconv_decoder | p(z) isotropic_gaussian | D = 10 | 2021-10-05-15:12:25
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
80 (inf-vae-exp 5 oct) INFO-VAE[l_1_rate=1000.0, l_2_mmd=100.0] | q(z|x) independent_gaussian | p(x|z) cond_pixel_cnn_pp | p(z) isotropic_gaussian | D = 10 | 2021-10-05-15:12:25
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)
(500, 10) (500,) (10,)


# Step 5: KNN prediction statistics

* For all reconstructions and samples:
instance_dists, marginal_pred
marginal_pred, uniform

In [12]:
from analysis.MNIST_analysis.analysis_steps import knn_prediction_distribution_stats

knn_prediction_distribution_stats(knn_data_pred_path="/home/cbarkhof/fall-2021/analysis/analysis-files/KNN_data_probas.p")

Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
No KNN predictions yet. Run knn_predictions_for_samples_reconstructions
/home/cbarkhof/fall-2021/analysis/analysis-files/data_group
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Saving KNN stats!
Savi

# Step 6: data space statistics

In [ ]:
from analysis.MNIST_analysis.analysis_steps import data_space_stats

data_space_stats()

knn_preds_recons dict_keys(['train', 'test', 'valid'])
knn_preds_cond_samples dict_keys(['train', 'test', 'valid'])
Saving data space stats!
knn_preds_recons dict_keys(['train', 'test', 'valid'])
knn_preds_cond_samples dict_keys(['train', 'test', 'valid'])
Saving data space stats!
knn_preds_recons dict_keys(['train', 'test', 'valid'])
knn_preds_cond_samples dict_keys(['train', 'test', 'valid'])
Saving data space stats!
knn_preds_recons dict_keys(['train', 'test', 'valid'])
knn_preds_cond_samples dict_keys(['train', 'test', 'valid'])
Saving data space stats!
knn_preds_recons dict_keys(['train', 'test', 'valid'])
knn_preds_cond_samples dict_keys(['train', 'test', 'valid'])
Saving data space stats!
knn_preds_recons dict_keys(['train', 'test', 'valid'])
knn_preds_cond_samples dict_keys(['train', 'test', 'valid'])
Saving data space stats!
knn_preds_recons dict_keys(['train', 'test', 'valid'])
knn_preds_cond_samples dict_keys(['train', 'test', 'valid'])
Saving data space stats!
knn_preds_rec